In [1]:
import pennylane as qml
from pennylane import numpy as np

## G.3.1

Concept: The **phase kickback trick**. The state $\ket{-}$ is an eigenstate of $X$, such that $X\ket{-}=-\ket{-}$. We exploit this property to construct a slightly modified $U_f$, with the assistance of an auxillary bit in state $\ket{-}$. For the simple case that $\vec{s}=\vec{1}$, we have $\mathrm{CNOT}_{1:n-1,n}\ket{\vec{1}-}=-\ket{\vec{1}-}$ and $\mathrm{CNOT}_{1:n-1,n}\ket{\vec{x}-}=\ket{\vec{x}-}$ for $\vec{x}\neq\vec{1}$. Then, to adapt this oracle into one for some fixed $\vec{s}$, sandwich the $\mathrm{CNOT}$ with a gate that performs the necessary $X$ transforms in the qubits where $\vec{s}$ is $\ket{0}$.

The nomenclature comes from conceptually separating the state into a **query register** of the first $n-1$ bits and the **auxillary bit**, the $n$-th bit. The $U_f$ changes the phase of the aux bit just in case our query register is the desired state, then conceptually we may reassociate ("kick back") this phase change into the query register by properties of phase with the tensor product.

In [3]:
n_bits = 5
query_register = list(range(n_bits))
aux = [n_bits]
all_wires = query_register + aux
dev = qml.device("default.qubit", wires=all_wires)


def oracle(combo):
    """Implement an oracle using a multi-controlled X gate.

    Args:
        combo (list): A list of bits representing the secret combination.
    """
    combo_str = "".join(str(j) for j in combo)
    ##################
    # YOUR CODE HERE #
    ##################
    qml.MultiControlledX(query_register, aux, combo_str)


## G.3.2

The **diffusion operator** can also be implemented utilizing the phase kickback trick, by defining $D=H_{1:n-1}\mathrm{CNOT}_{1:n-1,n}H_{1:n-1}$; the Hadamard transform converts the uniform superposition $\ket{\psi}$ to the $\ket{\vec{1}}$ vector, and any orthogonal component to $\ket{\psi}$ into some linear combination of the other basis vectors; and recall that $H$ is its own inverse.

In [ ]:
def hadamard_transform(my_wires):
    """Apply the Hadamard transform on a given set of wires.

    Args:
        my_wires (list[int]): A list of wires on which the Hadamard transform will act.
    """
    for wire in my_wires:
        qml.Hadamard(wires=wire)


def diffusion():
    """Implement the diffusion operator using the Hadamard transform and
    multi-controlled X."""

    ##################
    # YOUR CODE HERE #
    ##################
    hadamard_transform(query_register)
    qml.MultiControlledX(query_register, aux, "0" * len(query_register))
    hadamard_transform(query_register)


## G.3.3

In [ ]:
@qml.qnode(dev)
def grover_circuit(combo):
    """Apply the MultiControlledX Grover operator and return probabilities on
    query register.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[float]: Measurement outcome probabilities.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    # PREPARE QUERY AND AUXILIARY SYSTEM
    qml.PauliX(aux)
    hadamard_transform(all_wires)
    # APPLY GROVER ITERATION
    oracle(combo)
    diffusion()
    return qml.probs(wires=query_register)
